# CS918 - Assignment 1 - 1632704

First, load the libraries needed:

In [26]:
import json
import re
import nltk
from nltk.stem import WordNetLemmatizer
import timeit
import math
from collections import defaultdict

Time the code execution:

In [12]:
time_start = timeit.default_timer()

Read the dataset:

In [13]:
signal_news = []
with open('signal-news1.jsonl','r',encoding = 'utf-8') as f:
    for line in f:
        signal_news.append(json.loads(line))
del line

Keep only the data from the 'contents' section:

In [14]:
signal_news = [signal_news[i]['content'] for i in range(0,len(signal_news)-1)]

## Part A: Text Preprocressing

### 1. After lowercasing all the text, use regular expressions to parse and clean the texts:

In [15]:
signal_news = [word.lower() for word in signal_news]
signal_news = [word.replace('\n','') for word in signal_news]

- **Remove URLs. Note that URLs may appear in different forms, e.g. “http://www.*”, “http://domain”, “https://www.*”:**

In [16]:
signal_news = [re.sub(r'(?:https?\:\/\/)?(?:www\.)?[a-zA-Z0-9-]+\.(?:(?:[a-zA-Z0-9-]+\.)*)?[a-z]{2,4}(?:(?:\/\S+)*)?','',word) for word in signal_news]

The logic of the URL regex goes as follows: 

(?:https?:\/\/)? - non capture group with 1 or 0 occurences for the Hyper Text Transfer Protocol tag

(?:www\.)? - noon capture group with 1 or 0 occurences for www

[a-z0-9-]+\. - first part of websites name

(?:(?:[a-z0-9-]+\.)*)? - optional capture group to capture multi-worded domain names separated by dot (e.g. search.google.com) or names with multiple top level domains (e.g. data.gov.uk)

[a-z]{2,4} - final top level domain name, assumping it's no longer than 4 characters, but not shorter than 2 (which should usually be the case)

(?:(?:\/\S+)*)? - any futher possible combinations of characters separated by slashes

 - **Remove all non-alphanumeric characters except spaces, i.e. keep only alphanumeric characters and spaces:**

*1. Keep only alphanumeric - instance 1: for words such as 20-year-old, replace '-' with space:*

In [17]:
signal_news = [re.sub(r'(?:([a-z0-9])\-([a-z0-9]))+',r'\1 \2',word) for word in signal_news]

The logic of the above regex is as follows:
- Select at least one instance of a combination 'character-character' (for example '8-y' and 's-o' in '18-years-old')
- relace content between two capture groups with a space

*2. Keep only alphanumeric for other cases - simply remove all non-alphanumeric characters:*

In [18]:
signal_news = [re.sub(r'[^a-z0-9 ]','',word) for word in signal_news]

The above regex substitutes all characters that aren't letters, numbers or spaces to ''

**- Remove words with only 1 character:**

In [19]:
signal_news = [re.sub(r'\b(\w)\b','',word) for word in signal_news]

**- Remove numbers that are fully made of digits (e.g. you should remove the number ‘5’, but
in the case of ‘5pm’, made of both digits and letters, you should keep it as is, without
removing the digit that is part of the word):**

In [20]:
signal_news = [re.sub(r'\b\d+\b','',word) for word in signal_news]

As a final step, replace all the extra whitespaces created during pre-processing:

In [21]:
signal_news = [re.sub(r' +',' ',word) for word in signal_news]

### 2. Use an English lemmatiser to process all the words. Use of a POS tagger is optional, and you may instead assign each word the default POS tag when using the lemmatiser. [6 marks]

*2.1. Tokenize each text:*

In [24]:
signal_news_tokenized = [nltk.word_tokenize(word) for word in signal_news]

*2.2. Lemmatize using default POS tagger:*

In [29]:
lemmatizer = WordNetLemmatizer()
signal_news_lemmatized = signal_news_tokenized.copy()
for i in range(len(signal_news_tokenized)-1):
    signal_news_lemmatized[i] = [lemmatizer.lemmatize(text) for text in signal_news_tokenized[i]]

## Part B: N-Grams

**1. Compute N (number of tokens) and V (vocabulary size):**

In [30]:
#merge all vocabulary into one list:
signal_news_lemmatized_full = [word for text in signal_news_lemmatized for word in text]
#Compute:
print('The number of tokens is {}.'.format(len(signal_news_lemmatized_full)))
print('The vocabulary size is {}.'.format(len(set(signal_news_lemmatized_full))))

The number of tokens is 5745921.
The vocabulary size is 109420.


**2. List the top 25 trigrams based on the number of occurrences on the entire corpus:**

*2.1. Define a function for counting ngrams in text:*

In [31]:
def ngram_count(textdata,n):
    #1. list of lists of trigrams in each text
    textdata_trigrams = [list(nltk.ngrams(text,n)) for text in textdata]
    #2. join all into one list
    textdata_trigrams_all = []
    for elem in textdata_trigrams:
        textdata_trigrams_all.extend(elem)
        #3. count each:
    textdata_trigrams_count = {}
    for elem in textdata_trigrams_all:
        if elem not in textdata_trigrams_count:
            textdata_trigrams_count[elem] = 1
        else:
            textdata_trigrams_count[elem] += 1
    return textdata_trigrams_count

*2.2. Count the trigrams using the function defined above and sort them in descending order by count*

In [32]:
signal_news_trigrams_count = ngram_count(signal_news_lemmatized,3)
#Sort:
signal_news_trigrams_count_sorted = sorted(signal_news_trigrams_count.items(),key = lambda x: x[1], reverse = True)
#Get the top 25
for i, (trigram,number) in enumerate(signal_news_trigrams_count_sorted[0:25]):
    print('{}. Tirgram: {} occured {} times'.format(i+1,trigram,number))

1. Tirgram: ('one', 'of', 'the') occured 2430 times
2. Tirgram: ('on', 'share', 'of') occured 2093 times
3. Tirgram: ('day', 'moving', 'average') occured 1979 times
4. Tirgram: ('on', 'the', 'stock') occured 1566 times
5. Tirgram: ('a', 'well', 'a') occured 1420 times
6. Tirgram: ('in', 'research', 'report') occured 1415 times
7. Tirgram: ('in', 'research', 'note') occured 1373 times
8. Tirgram: ('for', 'the', 'quarter') occured 1221 times
9. Tirgram: ('the', 'united', 'state') occured 1218 times
10. Tirgram: ('the', 'year', 'old') occured 1216 times
11. Tirgram: ('average', 'price', 'of') occured 1193 times
12. Tirgram: ('research', 'report', 'on') occured 1177 times
13. Tirgram: ('research', 'note', 'on') occured 1138 times
14. Tirgram: ('share', 'of', 'the') occured 1132 times
15. Tirgram: ('the', 'end', 'of') occured 1129 times
16. Tirgram: ('in', 'report', 'on') occured 1124 times
17. Tirgram: ('earnings', 'per', 'share') occured 1121 times
18. Tirgram: ('buy', 'rating', 'to') occ

**3. Using the lists of positive and negative words provided with the corpus, compute the number of positive and negative word counts in the corpus:**

*3.1 Create two sets, one containing the positive words and one containing the negative words:*

In [33]:
#Create a set of positive words:
positive_words = []
with open('opinion-lexicon-English\\positive-words.txt','r') as p:
    positive_words.append(p.readlines())
positive_words = set([word.strip('\n,') for word in positive_words[0]])
#Create a set of negative words:
negative_words = []
with open('opinion-lexicon-English\\negative-words.txt','r') as n:
    negative_words.append(n.readlines())
negative_words = set([word.strip('\n,') for word in negative_words[0]])

FileNotFoundError: [Errno 2] No such file or directory: 'opinion-lexicon-English\\positive-words.txt'

*3.2. Second, join all the separate tokenized corpus lists in one long list:*

In [18]:
signal_news_lemmatized_all = []
for elem in signal_news_lemmatized:
    signal_news_lemmatized_all.extend(elem)

*3.3 Define a function for counting occurences:*

In [19]:
#Define utlity function for that purpose
#takes two arguments:
#textlist - tokenized corpus passed as a list
#wordset - set of words that are to be counted within the tokenized corpus
def count_words(textlist,wordset):
    words_count = {} #create empty dictionary to store occurences of each word
    for elem in textlist: #iterate over elements in tokenized list from corpus
        if elem not in words_count:
            words_count[elem] = 1 #create key-value pair if the word wasn't counted
        else:
            words_count[elem] += 1 #add one to value if the word already in the dictionary
    #sum the count of all words that are included in the wordset
    word_total = sum([words_count[k] for k in wordset if k in words_count.keys()])
    return word_total

The above function creates an empty dictionary and then populates it with words included in *wordset* that occured in *textlist* as keys and counts of their occurences in *textlist* as values. Then it returns total count of these words. The function is then applied to positive and negative word lexicons and the lematized corpus:

In [20]:
print('The total number of positive words in the corpus is {}.'.format(count_words(signal_news_lemmatized_all,positive_words)))
print('The total number of negative words in the corpus is {}.'.format(count_words(signal_news_lemmatized_all,negative_words)))

The total number of positive words in the corpus is 175361.
The total number of negative words in the corpus is 132257.


**4. Compute the number of news stories with more positive than negative words, as well as the number of news stories with more negative than positive words. News stories with a tie (same number of positive and negative words) should not be counted:**

To do that, the *count_words* function is used to create a comparison list - a list of length equal as the number of texts in the entire corpus, with positive values for more positive words and negative for more negative. Then the sum of positive and negative elements in the list is computed:

In [21]:
words_comparison = [count_words(elem, positive_words)-count_words(elem,negative_words) for elem in signal_news_lemmatized]
positive_article_count = len([k for k in words_comparison if k > 0])
print('There are {} articles with more positive than negative words'.format(positive_article_count))
negative_article_count = len([k for k in words_comparison if k < 0])
print('There are {} articles with more negative than positive words'.format(negative_article_count))

There are 10883 articles with more positive than negative words
There are 6346 articles with more negative than positive words


## Part C: Language Models

Splitting the data into training and test datasets:

In [22]:
signal_news_train = signal_news_lemmatized[:16000]
signal_news_test = signal_news_lemmatized[16001:]

To compute the Language Model for trigram, *LanguageModel* class was created. It consists of:
1. ***__init__ ***method, which initializes an instance of the class by assigning the training data and basic model as its main attributes.

2. ***compute_model_trigrams*** method which uses two nested collections.defaultdict objects populated with 1s (for the purpose of Add-1 smoothing in preplexity calculation). Each bigram (w1,w2) appearing in the text is associated with frequencies of unigrams w3 occuring after it.

3. ***compute_model_bigrams*** method used for back-off in preplexity calculation. Computed by analogy to the method above, only this time calculates frequencies of unigrams w2 appearing after each unigram w1 in the trainin data.

4. ***compute_model_unigrams*** method used for back-off in preplexity calculation. By analogy to the two above, only this time calculates unigram model, i.e. frequencies of each word.

5. ***generate_sentence*** method which takes two initial words and sequentially predicts *i* words occuring next by selecting those with highest conditional frequency from the trigram model.

6. ***compute_denominators*** method, which calculates the denominators used for normalization of Maximum Likelihood Estimates of conditional word probabilities after applying add-1 smoothing. Denominator for each observed ngram is equal to the sum of frequencies associated with it + the vocabulary size for the ngram (possible words following it).

7. ***compute_perplexity*** method, which, given a *test_set* performs intrinsic evaluation of the fitted model. It does so by assigning each word w3 following a bigram (w1,w2) in the unobserved data with log of probability predicted by the trained model. These probabilities are obtained by dividing the add-1 frequency by the normalized denominator computed by the *compute_denominator* method. In case of a bigram not being observed by the trigram model, it tries to use a bigram and then a unigram. As a last resort, it estimates the probability as 1 over the number of unique bigrams in the entire corpusus. Finally, the perplexity is given by $e^{-\frac{1}{n}log(w_{1},w_{2},...,w_{n})}$

In [23]:
class LanguageModel:
    def __init__(self,train_data):
        self.data = train_data
        self.model_trigrams = self.compute_model_trigrams()
        self.model_bigrams = self.compute_model_bigrams()
        self.model_unigrams = self.compute_model_unigrams()

        
    def compute_model_trigrams(self):
        model_trigrams = defaultdict(lambda:defaultdict(lambda:1))
        for text in self.data:
            for w1, w2, w3 in nltk.ngrams(text,3):
                model_trigrams[(w1,w2)][w3] += 1
        return model_trigrams
    
    def compute_model_bigrams(self):
        model_bigrams = defaultdict(lambda:defaultdict(lambda:1))
        for text in self.data:
            for w1, w2 in nltk.ngrams(text,2):
                model_bigrams[w1][w2] += 1
        return model_bigrams
    
    def compute_model_unigrams(self):
        model_unigrams = defaultdict(lambda:0)
        for text in self.data:
            for w1 in nltk.ngrams(text,1):
                model_unigrams[w1] += 1
        return model_unigrams
    
    def generate_sentence(self,word1,word2,i):
        sentence = ''.join([word1, ' ',word2])
        while i > 0:
            words = dict(self.model_trigrams[word1,word2])
            word1 = word2
            word2 = max(words,key = words.get)
            sentence = sentence + ' ' + word2
            i -= 1
        return(sentence)
    
    def compute_denominators(self,model):
        denominators = dict()
        for w in model:
            denominator = int((sum(model[w].values())) + len(model[w]))
            denominators[w] = denominator
        return denominators

    
    def compute_perplexity(self,test_data):
        denominators_trigrams = self.compute_denominators(self.model_trigrams)
        denominators_bigrams = self.compute_denominators(self.model_bigrams)
        denominators_unigrams = float(sum(self.model_unigrams.values()) + len(ngram_count(self.data,1))) #total count + count of unique unigrams
        unique_bigrams = len(ngram_count(self.data,2))
        trigrams_test = []
        test_data_full = []
        for text in test_data:
            test_data_full.extend(text)
        N = len(set(test_data_full))
        for text in test_data:
            trigram = [(w1,w2,w3) for w1,w2,w3 in nltk.ngrams(text,3)]
            trigrams_test.append(trigram)
        #set log_prob to 1
        log_prob = 1
        for trigram in trigrams_test:
            for w1,w2,w3 in trigram:
                #first - try to get probability from the smoothed trigram model:
                if (w1,w2) in self.model_trigrams.keys():
                    #ideally the if statement would be sufficient, however errors occured -> exception handling necessary
                    try:
                        log_prob += math.log(self.model_trigrams[(w1,w2)][w3]/denominators_trigrams[(w1,w2)])
                    except:
                        #if the bigram did not appear in the trigram model, try a bigram model for the second word:  
                        if (w1,w2) in self.model_bigrams.keys():
                            log_prob += math.log(self.model_bigrams[w2][w3]/denominators_bigrams[w2])
                        #otherwise, try estimating using unigram
                        else:
                            try:
                                log_prob += math.log(self.model_unigrams[w3]/denominators_unigrams)
                            #last resort: estimate probability as 1/count of unique bigrams in the entire corpus
                            except:
                                log_prob += math.log(self.model_trigrams[(w1,w2)][w3]/unique_bigrams)
        perplexity = math.e**(-(1/N)*log_prob)
        return perplexity

The following code initializes the Language Model:

In [24]:
lm = LanguageModel(signal_news_train)

**1. Compute language models for trigrams based on the first 16,000 rows of the corpus. Beginning with the bigram “is this”, produce a sentence of 10 words by appending the most likely next word each time.**

The following line of code predicts a 10 word sentence beginning with a bigram ('is','this'):

In [31]:
lm.generate_sentence('do','something',20)

'do something about it the first time in the first time in the first time in the first time in the first'

**2. Compute the perplexity by evaluating on the remaining rows of the corpus (rows 16,001+).**

In [ ]:
lm.compute_perplexity(signal_news_test)

The perplexity is extremely high, which points at poor performance of the created model.

In [ ]:
time_end = timeit.default_timer()
print(time_end-time_start)